In [ ]:
import os
import pyspark
conf = pyspark.SparkConf()

conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

In [ ]:
from pyspark.sql.functions import *
import pandas as pd

In [ ]:
#Q2

In [ ]:
#A
df = spark.read.option("inferSchema", "true").option("header", "true").csv("shared/data/midterm/drive_stats_2019_Q1")

In [ ]:
df.describe()

In [ ]:
#B
col = list(df.columns)
print(col)

In [ ]:
#C
ct = df.select("model").distinct().count()
print(ct)

In [ ]:
#D
df.select("model", "capacity_bytes").distinct().sort('capacity_bytes',ascending=False)

In [ ]:
#E
df.select("model", "serial_number").distinct().count()

In [ ]:
#E
df.select("model", "serial_number").distinct()

In [ ]:
#F
df.select('smart_200_normalized').describe()

In [ ]:
#G
df.crosstab('model', 'smart_200_normalized')

In [ ]:
#H
df.select('capacity_bytes').describe()

In [ ]:
#I 
df.sort('smart_200_normalized',ascending=False).limit(10)

In [ ]:
#J
dfj = df.filter("smart_200_normalized is Null" or "smart_200_normalized==0")
dfj.select('model','capacity_bytes')

In [ ]:
#Q3

In [ ]:
#A

df3 = df.filter(df.smart_200_normalized.isNotNull())
df3 = df3.select("model","smart_200_normalized")
df3.select('smart_200_normalized').describe()


In [ ]:
df3.createOrReplaceTempView('filter_view')

In [ ]:
df3 = spark.sql("""select * from filter_view where (smart_200_normalized>98.9626451182) and (smart_200_normalized<101.056082017)""")
df3.groupBy("model","smart_200_normalized").count().show()

In [ ]:
#B

In [ ]:
df.createOrReplaceTempView('filter_view2')

In [ ]:
sq1=spark.sql("""SELECT model, count(*) AS upTime FROM filter_view2 GROUP BY model;""")
sq2=spark.sql("""SELECT model, count(*) AS failures FROM filter_view2 WHERE failure = 1 GROUP BY model;""")

In [ ]:
sq2=sq2.withColumnRenamed("model","model2")
joinDF=sq1.join(sq2,sq1.model == sq2.model2,"leftouter")
joinDF=joinDF.drop("model2")
joinDF=joinDF.na.fill(value=0,subset=["failures"])
joinDF=joinDF.withColumn("ErrorRate",((joinDF["failures"]/joinDF["upTime"])*100*365))

std4=joinDF

dfstats=std4.select("ErrorRate")
std4=dfstats.summary("stddev","mean")
temp=std4.rdd.map(lambda x: x.ErrorRate).collect()


std=float(temp[0])
mean=float(temp[1])

lower= mean - std
upper= mean + std


In [ ]:
from pyspark.sql import functions as func
finalDF=joinDF.withColumn("Anomaly",func.when((joinDF["ErrorRate"]>=lower) & (joinDF["ErrorRate"]<=upper),False).otherwise(True))
finalDF
